In [ ]:
import os
import random
import warnings

warnings.filterwarnings("ignore")

import lpips
import yaml

from tqdm.auto import tqdm

import numpy as np
import pandas as pd

import torch
from diffusers import StableDiffusionPipeline, LMSDiscreteScheduler
from transformers import CLIPProcessor, CLIPModel
from torchvision import transforms

from PIL import Image

In [5]:
def delete_pipeline(pipeline):
    del pipeline.vae
    del pipeline.tokenizer
    del pipeline.text_encoder
    del pipeline.unet
    del pipeline
    torch.cuda.empty_cache()

def print_cuda():
    gpu_count = torch.cuda.device_count()
    for gpu_idx in range(gpu_count):
        print(f"GPU {gpu_idx}: {torch.cuda.get_device_name(gpu_idx)}")
        print(f"Memory Allocated: {torch.cuda.memory_allocated(gpu_idx) / 1024 / 1024:.2f} MB")
        print(f"Memory Reserved: {torch.cuda.memory_reserved(gpu_idx) / 1024 / 1024:.2f} MB")
    print()

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [ ]:
@torch.no_grad()
def generateImages(pipeline, prompts, prompt_path, save_path,
                   sample_count=5, guidance_scale=7.5,):
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    config = {
        "sample_count": sample_count,
        "prompt_count": len(prompts),
        "guidance_scale": guidance_scale
    }
    with open(f"{save_path}/config.yaml", 'w') as file:
        yaml.dump(config, file)

    device = pipeline.unet.device

    if prompt_path:
        df = pd.read_csv(prompt_path)
        prompts = df.prompt
        seeds = df.seed
    else:
        seeds = [random.randint(0, 5000) for _ in range(len(prompts))]

    for idx, prompt in enumerate(tqdm(prompts)):

        scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)
        scheduler.set_timesteps(100)

        prompts_ = [prompt] * sample_count + [""] * sample_count

        tokens = pipeline.tokenizer(prompts_, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids.to(device)
        embds = pipeline.text_encoder(tokens)[0]

        generator = torch.Generator(device=device).manual_seed(int(seeds[idx]))
        latents = torch.randn((sample_count, 4, 64, 64), generator=generator).to(device)
        latents *= scheduler.init_noise_sigma

        for t in scheduler.timesteps:

            latent_inputs = torch.cat([latents] * 2)
            latent_inputs = scheduler.scale_model_input(latent_inputs, timestep=t)

            noises = pipeline.unet(latent_inputs, t, encoder_hidden_states=embds).sample
            cond_noises, uncond_noises = noises.chunk(2)
            noises = uncond_noises + guidance_scale * (cond_noises - uncond_noises)

            latents = scheduler.step(noises, t, latents).prev_sample

        latents /= 0.18215
        images = pipeline.vae.decode(latents).sample
        images = ((images + 1) / 2).clamp(0, 1).permute(0, 2, 3, 1)
        images = images.detach().cpu().numpy()
        images = (images * 255).round().astype("uint8")
        for idx_, image in enumerate(images):
            image = Image.fromarray(image)
            image.save(f"{save_path}/{idx}_{idx_}.png")

    df = pd.DataFrame({"prompt": prompts, "seed": seeds})
    df.to_csv(f"{save_path}/info.csv")

In [32]:
type(int(pd.read_csv("data/default_1/info.csv").seed[0]))

int

In [34]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipeline = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base")
pipeline.unet.to(device).half()
pipeline.vae.to(device).half()

save_path = "data/default_1"

df = pd.read_csv("data/artists1734_prompts.csv")
artists = list(df.artist.unique())
random.shuffle(artists)
prompts = artists[:20]
generateImages(pipeline, prompts, None, save_path)

delete_pipeline(pipeline)
print_cuda()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

RuntimeError: expected mat1 and mat2 to have the same dtype, but got: float != struct c10::Half

In [37]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
pipeline = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base")
pipeline.unet.to(device)
pipeline.vae.to(device)

lamb = 0.5
eraseScale = 1
preserveScale = 0.1
withKey = False

erased_pipeline = eraseModel(pipeline, prompts[:10], ["art"]*10, prompts[10:], lamb, eraseScale, preserveScale, withKey, "data/artist10erase_1")
generateImages(pipeline, prompts, "data/default_1/info.csv", "data/artist10erase_1")

delete_pipeline(pipeline)
print_cuda()

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [17]:
@torch.no_grad()
def eraseModel(model, prevPrompts, newPrompts, retainPrompts=None, 
               lamb=0.1, eraseScale=0.1, preserveScale=0.1, withKey=True,
               name="erase"):

    """
    prevPrompts에 해당하는 개념을 newPrompts로 편집한 모델 반환.

    Parameters:
        model (StableDiffusionPipeline): 편집할 모델.
        prevPrompts (List[str]): 원래 개념이 저장된 string 리스트.
        newPrompts (List[str]): 대체할 개념이 저장된 string 리스트.
        retainPrompts (List[str] | None): 보존할 개념이 저장된 string 리스트. Default: None
        lamb (float): 정규화 강도. Default: 0.1
        eraseScale (float): 개념 편집 강도. Default: 0.1
        preserveScale (float): 개념 보존 강도. Default: 0.1
        withKey (bool): key 가중치 업데이트 여부. Default: True

    Returns:
        model (StableDiffusionPipeline)
    """

    device = model.unet.device

    # 주어진 모델 unet의 cross-attention layer를 caLayers에 저장.
    caLayers = []
    for name, module in model.unet.named_modules():
        # attn2로 끝나는 모듈이 cross-attention layer.
        if name[-5:] != "attn2": continue
        caLayers.append(module)

    # value projection layer를 targetLayers에 저장. 논문의 W^old에 해당하는 부분.
    # (value projection layer): Linear(in_features=1024, out_features=320, bias=False)
    # "stabilityai/stable-diffusion-2-1-base" 모델이 기준. "CompVis/stable-diffusion-v1-4" 모델은 1024 대신 768을 사용.
    valueLayers = [layer.to_v for layer in caLayers]
    targetLayers = valueLayers

    # withKey=True라면 key projection layer도 추가.
    if withKey: 
        # (key projection layer): Linear(in_features=1024, out_features=320, bias=False)
        keyLayers = [layer.to_k for layer in caLayers]
        targetLayers += keyLayers
    
    # 텍스트 prevPrompts를 텍스트 임베딩 prevEmbds으로 변환. prevEmbds의 원소는 논문의 c_i에 대응됨.
    # (prevPrompts): (N,)
    # ex) N = 2; prevPrompts = ['Kelly Mckernan', 'Sarah Anderson']
    # (prevEmbds): (N, 1024, 77)
    prevInputs = model.tokenizer(prevPrompts, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids
    prevEmbds = model.text_encoder(prevInputs)[0].permute(0, 2, 1).to(device)


    # 마찬가지로 newEmbds 생성. newEmbds의 원소는 논문의 c*_i에 대응됨.
    # (newPrompts): (N,)
    # ex) newEmbds = ['art', 'art']
    # (newEmbds): (N, 1024, 77)
    newInputs = model.tokenizer(newPrompts, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids
    newEmbds = model.text_encoder(newInputs)[0].permute(0, 2, 1).to(device)

    # 논문에서 제시한 closed-form solution은 W = [sum{(W^old)(c*_i)(c_i)^T}+lambda*(W^old)][sum{(c_i)(c_i)^T)}+lambda*(I)]^(-1).
    # W의 첫번째 대괄호 부분이 m1, 두번째 대괄호가 m2. 즉 W = [m1][m2]^(-1).

    # m1 = (W^old)[sum{(c*_i)(c_i)^T}+lambda*(I)].
    # m1의 대괄호 부분이 m3. 즉 m1 = (W^old)[m3].

    # m2 = [sum{(c_i)(c_i)^T)}+lambda*(I)].
    # (m2): (1024, 1024)
    m2 = (prevEmbds @ prevEmbds.permute(0, 2, 1)).sum(0) * eraseScale
    m2 += lamb * torch.eye(m2.shape[0], device=device)

    # m3 = [sum{(c*_i)(c_i)^T}+lambda*(I)].
    # (m3): (1024, 1024)
    m3 = (newEmbds @ prevEmbds.permute(0, 2, 1)).sum(0) * eraseScale
    m3 += lamb * torch.eye(m3.shape[0], device=device)

    # retainPrompts가 있다면 m2와 m3에 sum{(c_j)(c_j)^T} 추가
    if retainPrompts:
        # retainEmbds 생성. retainEmbds의 원소는 논문의 c*_j에 대응됨.
        # (retainPrompts): (M,)
        # (retainEmbds): (M, 1024, 77)
        retainInputs = model.tokenizer(retainPrompts, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids
        retainEmbds = model.text_encoder(retainInputs)[0].permute(0, 2, 1).to(device)

        m2 += (retainEmbds @ retainEmbds.permute(0, 2, 1)).sum(0) * preserveScale
        m3 += (retainEmbds @ retainEmbds.permute(0, 2, 1)).sum(0) * preserveScale

    for targetLayer in targetLayers:
        # (m1): (320, 1024)
        # (targetLayer.weight): (320, 1024)
        m1 = targetLayer.weight @ m3
        targetLayer.weight = torch.nn.Parameter((m1 @ torch.inverse(m2)).detach())
    
    torch.save(model.unet.state_dict(), f"model/{name}.pt")

    config = {
        "lamb": lamb,
        "erase_scale": eraseScale,
        "erase_scale": preserveScale,
        "preserve_scale": preserveScale,
        "withKey": withKey
    }
    with open(f"model/{name}.yaml", 'w') as file:
        yaml.dump(config, file)

    return model

In [3]:
eraseArtistCount = 10
lamb = 0.5
eraseScale = 1
preserveScale = 0.1
withKey = False

stepCount = 100
guidanceScale = 7.5

setSeed(2)

with torch.no_grad():

    df = pd.read_csv("data/artists1734_prompts.csv")
    artists = list(df.artist.unique())
    random.shuffle(artists)

    eraseArtists = artists[:eraseArtistCount]
    retainArtists = artists[eraseArtistCount:]

    prefixes = [""]

    prevPrompts = []
    newPrompts = []
    retainPrompts = []
    for prefix in prefixes:
        for eraseArtist in eraseArtists:
            prevPrompts.append(prefix + eraseArtist)
            newPrompts.append(prefix + "art")
        for retainArtist in retainArtists:
            retainPrompts.append(prefix + retainArtist)
    retainPrompts = retainPrompts[:eraseArtistCount]
    prompts = prevPrompts + retainPrompts
    if len(retainPrompts) == 0: retainPrompts = None

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base").to(device)

    vae, tokenizer, textEncoder, unet = model.vae, model.tokenizer, model.text_encoder, model.unet

    scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

    seeds = [random.randint(0, 5000) for idx in range(len(prompts))]

    for idx, prompt in enumerate(prompts):

        prompt = [prompt] + [""]
        input = tokenizer(prompt, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids.to(device)
        embd = textEncoder(input)[0]

        seed = seeds[idx]
        generator = torch.Generator()
        generator.manual_seed(seed)
        latent = torch.randn((1, unet.in_channels, 64, 64), generator=generator).to(device)
        latent *= scheduler.init_noise_sigma
        
        scheduler.set_timesteps(stepCount)
        for t in tqdm.tqdm(scheduler.timesteps):
            latentInput = torch.cat([latent] * 2)
            latentInput = scheduler.scale_model_input(latentInput, timestep=t)

            noise = unet(latentInput, t, encoder_hidden_states=embd).sample
            condNoise, uncondNoise = noise.chunk(2)
            noise = uncondNoise + guidanceScale * (condNoise - uncondNoise)

            latent = scheduler.step(noise, t, latent).prev_sample

        latent /= 0.18215
        image = vae.decode(latent).sample[0]
        image = ((image + 1) / 2).clamp(0, 1).permute(1, 2, 0)
        image = image.detach().cpu().numpy()
        image = (image * 255).round().astype("uint8")
        image = Image.fromarray(image)
        image.save(f"image/default/{idx}.png")

    erasedModel = eraseModel(model=model, prevPrompts=prevPrompts, newPrompts=newPrompts, retainPrompts=retainPrompts,
                            lamb=lamb, eraseScale=eraseScale, preserveScale=preserveScale, withKey=withKey)
    unet = erasedModel.unet

    torch.cuda.empty_cache()

    for idx, prompt in enumerate(prompts):

        prompt = [prompt] + [""]
        input = tokenizer(prompt, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids.to(device)
        embd = textEncoder(input)[0]

        seed = seeds[idx]
        generator = torch.Generator()
        generator.manual_seed(seed)
        latent = torch.randn((1, unet.in_channels, 64, 64), generator=generator).to(device)
        latent *= scheduler.init_noise_sigma
        
        scheduler.set_timesteps(stepCount)
        for t in tqdm.tqdm(scheduler.timesteps):
            latentInput = torch.cat([latent] * 2)
            latentInput = scheduler.scale_model_input(latentInput, timestep=t)

            noise = unet(latentInput, t, encoder_hidden_states=embd).sample
            condNoise, uncondNoise = noise.chunk(2)
            noise = uncondNoise + guidanceScale * (condNoise - uncondNoise)

            latent = scheduler.step(noise, t, latent).prev_sample

        latent /= 0.18215
        image = vae.decode(latent).sample[0]
        image = ((image + 1) / 2).clamp(0, 1).permute(1, 2, 0)
        image = image.detach().cpu().numpy()
        image = (image * 255).round().astype("uint8")
        image = Image.fromarray(image)
        image.save(f"image/erase/{idx}.png")

    df = pd.DataFrame({"prompt": prompts, "seed": seeds})
    df.to_csv("image/df.csv")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:30<00:00,  3.26it/s]


In [8]:
import pandas as pd
import random
eraseArtistCount = 10
lamb = 0.5
eraseScale = 1
preserveScale = 0.1
withKey = False

stepCount = 100
guidanceScale = 7.5

with torch.no_grad():

    df = pd.read_csv("data/artists1734_prompts.csv")
    artists = list(df.artist.unique())
    random.shuffle(artists)

    eraseArtists = artists[:eraseArtistCount]
    retainArtists = artists[eraseArtistCount:]


eraseArtists[0]

'Anton Domenico Gabbiani'

In [39]:
def loadImages(folderPath, scale=True):
    
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    images = []
    for idx in range(len(os.listdir(folderPath))):
        image = Image.open(f"{folderPath}/{idx}.png").convert('RGB')
        image = preprocess(image)
        images.append(image)
    
    images = torch.stack(images)
    if scale: images = images * 2 - 1
    return images


loss_fn = lpips.LPIPS(net='alex')

images = loadImages("image/default")
erasedImages = loadImages("image/erase")

pd.DataFrame({"loss":loss_fn(images, erasedImages).squeeze().detach().numpy().round(3)})

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\yoonj\AppData\Local\Programs\Python\Python312\Lib\site-packages\lpips\weights\v0.1\alex.pth


,loss
0,0.721
1,0.723
2,0.656
3,0.688
4,0.644
5,0.665
6,0.603
7,0.759
8,0.737
9,0.632


In [43]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

images = loadImages("image/default", False)
inputs = processor(text=prompts, images=images, return_tensors="pt", padding=True, do_rescale=False)

with torch.no_grad():
    outputs = model(**inputs)
    imageEmbds = outputs.image_embeds
    textEmbds = outputs.text_embeds

score = torch.nn.functional.cosine_similarity(imageEmbds, textEmbds)
print("CLIP Score:", score)

CLIP Score: tensor([0.3543, 0.3257, 0.3125, 0.3000, 0.3022, 0.3020, 0.3057, 0.3338, 0.3074,
        0.3147, 0.3050, 0.2925, 0.3142, 0.3031, 0.2479, 0.2718, 0.3003, 0.3175,
        0.3061, 0.2635])


In [44]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

images = loadImages("image/erase", False)
inputs = processor(text=prompts, images=images, return_tensors="pt", padding=True, do_rescale=False)

with torch.no_grad():
    outputs = model(**inputs)
    imageEmbds = outputs.image_embeds
    textEmbds = outputs.text_embeds

score = torch.nn.functional.cosine_similarity(imageEmbds, textEmbds)
print("CLIP Score:", score)

CLIP Score: tensor([0.1301, 0.1472, 0.1725, 0.1093, 0.2160, 0.1015, 0.2042, 0.2288, 0.1318,
        0.2088, 0.3226, 0.3118, 0.2988, 0.3204, 0.2522, 0.2586, 0.2815, 0.3177,
        0.3096, 0.2714])


In [42]:
imageEmbds.shape

torch.Size([20, 512])

In [31]:
from clip_score import get_clip_score

folderPath = "image/default"
imagePaths = [f"{folderPath}/{idx}.png" for idx in range(len(os.listdir(folderPath)))]
scores = get_clip_score(imagePaths, prompts)

for i, prompt in enumerate(prompts):
    print(f"Scores for text: '{prompt}'")
    for j, imagePath in enumerate(imagePaths):
        print(f"  {os.path.basename(imagePath)}: {scores[j][i]}")

ImportError: cannot import name 'get_clip_score' from 'clip_score' (c:\Users\yoonj\AppData\Local\Programs\Python\Python312\Lib\site-packages\clip_score\__init__.py)

In [3]:
#erase
if __name__ == "__main__":
    
    ################################################################################################################################

    eraseArtistCount = 10
    lamb = 0.5
    eraseScale = 1
    preserveScale = 0.1
    withKey = False

    ################################################################################################################################

    df = pd.read_csv("data/artists1734_prompts.csv")
    artists = list(df.artist.unique())
    random.shuffle(artists)

    eraseArtists = artists[:eraseArtistCount]
    retainArtists = artists[eraseArtistCount:]

    prompts = ["painting by ", "art by ", "artwork by ", "picture by ", "style of ", ""]
    prompts = [""]

    prevPrompts = []
    newPrompts = []
    retainPrompts = []
    for prompt in prompts:
        for eraseArtist in eraseArtists:
            prevPrompts.append(prompt + eraseArtist)
            newPrompts.append(prompt + "art")
        for retainArtist in retainArtists:
            retainPrompts.append(prompt + retainArtist)
    if len(retainPrompts) == 0: retainPrompts = None

    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    model = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base").to(device)
    erasedModel = eraseModel(model=model, prevPrompts=prevPrompts, newPrompts=newPrompts, retainPrompts=retainPrompts,
                             lamb=lamb, eraseScale=eraseScale, preserveScale=preserveScale, withKey=withKey)

    

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

NameError: name 'eraseModel' is not defined

In [3]:
#moderate
if __name__ == "__main__":

    concept_type = "unsafe"
    concepts = "violence, nudity, harm"
    guided_concepts = None
  
    concepts = [c.strip() for c in concepts.split(',')]

    if concept_type == "art":
        prompts = ["painting by ", "art by ", "artwork by ", "picture by ", "style of ", ""]
    elif concept_type == "object":
        prompts = ["image of ", "photo of ", "portrait of ", "picture of ", "painting of ", ""]
    else:
        prompts = [""]

    prevPrompts = []
    for concept in concepts:
        for prompt in prompts:
            prevPrompts.append(prompt + concept)

    newPrompts = []
    if guided_concepts:
        for concept in [guided_concepts] * len(concepts):
            for prompt in prompts:
                newPrompts.append(prompt + concept)
    else:
        newPrompts = [' '] * len(prevPrompts)
    
    lamb = 0.5
    device = "cuda:0" if torch.cuda.is_available() else "cpu"

    model = StableDiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base")
    model = model.to(device)
    model = eraseModel(model=model, prevPrompts=prevPrompts, newPrompts=newPrompts, lamb=lamb,)

    torch.save(model.unet.state_dict(), "model/moderate.pt")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [2]:
@torch.no_grad()
def debiasModel(model, prevPrompts, newPrompts, lamb=0.1, scale=0.1, withKey=True):
    
    """
    prevPrompts에 해당하는 concept이 newPrompts에 해당하는 attribute에 대하여 debiasing된 모델 반환.

    Parameters:
        model (StableDiffusionPipeline): debias할 모델.
        prevPrompts (List[str]): debias할 concept을 저장하고 있는 텍스트 리스트.
        newPrompts (List[List[str]]): debias 대상 attribute를 저장하고 있는 텍스트 리스트의 리스트
        lamb (float): 정규화 강도. Default: 0.1
        scale (float): debiasing 강도. Default: 0.1
        withKey (bool): key weight 업데이트 여부. Default: True

    Returns:
        model (StableDiffusionPipeline)
    """

    device = model.device

    # SD 모델 unet의 모든 cross-attention layer를 caLayers에 저장.
    caLayers = []
    for name, module in model.unet.named_modules():
        # attn2가 cross-attention를 의미함.
        if name[-5:] != "attn2": continue
        caLayers.append(module)

    # cross-attention layer의 value 부분을 valueLayers에 저장.
    valueLayers = [layer.to_v for layer in caLayers]
    # withKey 옵션이 켜져 있다면 key 부분도 추가함.
    if withKey: valueLayers + [layer.to_k for layer in caLayers]

    # 텍스트 리스트인 prevPrompts를 텍스트 임베딩인 prevEmbds으로 변환.
    prevInputs = model.tokenizer(prevPrompts, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids.to(device)
    # prevEmbds는 (N, 768, 77) 형태이며 각 임베딩은 논문의 c_i에 대응됨. (N은 prevPrompts의 길이)
    prevEmbds = model.text_encoder(prevInputs)[0].permute(0, 2, 1)

    # eraseModel과 거의 같음.
    # 다만 m3 = sum{(c_i^*)(c_i)^T}+lambda*(I)의 c_i^* 부분 대신 [(c_i)+sum{alpha/|(W_old)(a)|*|(W_old)(c_i)|*(a)}]이 사용됨.
    # (a는 newPrompts의 원소인 newPrompt의 각 임배딩, alpha는 각 임베딩의 가중치)

    # m2 = sum((c_i)(c_i)^T)}+lambda*(I)
    m2 = (prevEmbds @ prevEmbds.permute(0, 2, 1)).sum(0) * scale
    m2 += lamb * torch.eye(m2.shape[1], device=device)

    clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    generator = torch.Generator(device=device)

    # ratio 차이가 alpha에 반영되는 비율.
    eta = 0.1
    threshold = 0.05
    alphas = [torch.zeros(len(t)) for t in newPrompts]
    targetRatios = [torch.ones(len(t)) / len(t) for t in newPrompts]
    # 각 prevPrompt에 해당하는 alpha의 업데이트가 필요한지 나타냄.
    check = [0] * len(prevPrompts)
    for _ in range(30):
        for idx in range(len(prevPrompts)):

            # alpha를 업데이트하지 않고 모두 0으로 바꾸어 value weight 업데이트가 일어나지 않도록 함.
            if check[idx]: 
                alphas[idx] *= 0
                continue
        
            prevPrompt = prevPrompts[idx]
            newPrompt = newPrompts[idx]

            # SD model로 prevPrompt에 해당하는 이미지 50개 생성
            images = model(prevPrompt, num_images_per_prompt=50, num_inference_steps=20, generator=generator).images
            
            # score는 생성된 이미지 50개와 텍스트 리스트인 newPrompt 사이의 유사도 점수로 (50, M) 형태. (M은 newPrompt의 길이)
            score = clip_model(**clip_processor(text=newPrompt, images=images, return_tensors="pt", padding=True)).logits_per_image

            # 각 이미지에 대해 가장 높은 유사도를 가진 점수를 1. 나머지를 0.으로 변환하여 각 이미지에 대해 평균을 냄.
            # 즉 ratio는 prevPrompt로 생성된 이미지가 newPrompt의 각 prompt에 해당할 확률을 나타냄.
            ratio = score.ge(score.max(1)[0].view(-1,1)).float().mean(0)
            # 각 prompt에 해당할 확률이 동일하기를 원하기 때문에 targetRatio와의 차이가 반영 정도가 됨.
            alpha = (eta * (targetRatios[idx] - ratio)).to(device)
            alphas[idx] = alpha

            # ratio와 targetratio와의 차이가 threshold 보다 작다면 더 이상 업데이트가 필요하지 않음.
            if ratio.abs().max() < threshold: check[idx] = 1

        # 모든 prompt에 대해서 업데이트가 필요하지 않다면 루프를 종료함.
        if sum(check) == len(prevPrompts): break

        # [(c_i)+sum{alpha/|(W_old)(a)|*|(W_old)(c_i)|*(a)}]
        for valueLayer in valueLayers:
            reEmbds = []
            for idx in range(len(prevPrompts)):
        
                alpha = alphas[idx]
                newPrompt = newPrompts[idx]
                prevEmbd = prevEmbds[idx]

                newInput = model.tokenizer(newPrompt, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids.to(device)
                # newEmbd은 (M, 768, 77) 형태의 텍스트 임베딩.
                newEmbd = model.text_encoder(newInput)[0].permute(0, 2, 1)
                # norm은 (M,) 형태의 벡터로 alpha/|(W_old)(a)|*|(W_old)(c_i)|를 나타냄.
                norm = alpha / (valueLayer.weight @ newEmbd).norm(dim=[1,2]) * (valueLayer.weight @ prevEmbd).norm()
                
                # reEmbd은 (768, 77) 형태의 텍스트 임베딩으로 c_i + sum{alpha/|(W_old)(a)|*|(W_old)(c_i)|*(a)}를 나타냄.
                reEmbd = prevEmbd + (norm.view(-1, 1, 1) * newEmbd).sum(0)
                reEmbds.append(reEmbd.unsqueeze(0))
            reEmbds = torch.concat(reEmbds, 0)

            m3 = (reEmbds @ prevEmbds.permute(0, 2, 1)).sum(0) * scale
            m3 += lamb * torch.eye(m3.shape[1], device=device)
            # m1 = (W_old)[m3]
            m1 = valueLayer.weight @ m3
            valueLayer.weight = torch.nn.Parameter((m1 @ torch.inverse(m2)).detach())
        
    return model

# debias
if __name__ == "__main__":

    concepts = "Doctor, Nurse, Carpenter"
    attributes = "male, female"

    concepts = [c.strip() for c in concepts.split(',')]
    attributes = [a.strip() for a in attributes.split(',')]

    prompts = ["image of ", "photo of ", "portrait of ", "picture of ", ""]
    
    prevPrompts = []
    newPrompts = []
    for prompt in prompts:
        for concept in concepts:
            prevPrompts.append(prompt + concept)
            newPrompt = []
            for attribute in attributes:
                newPrompt.append(prompt + attribute)
            newPrompts.append(newPrompt)

    lamb = 0.5
    device = "cuda:0" if torch.cuda.is_available() else "cpu"

    model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4")
    model = model.to(device)
    model = debiasModel(model=model, prevPrompts=prevPrompts, newPrompts=newPrompts, lamb=lamb,)

    torch.save(model.unet.state_dict(), "model/debias.pt")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [7]:
@torch.no_grad()
def generateImage(promptPath, modelVersion="1.4", sampleCount=10, imageSize=(512, 512), stepCount=100, guidanceScale=7.5):
    
    """
    주어진 prompt 파일에 해당하는 이미지를 생성하여 "image" 폴더에 저장.

    Parameters:
        promptPath (str): prompt 파일이 저장된 경로.
        modelVersion (str): 이미지 생성시 사용되는 모델의 버전. Default: "1.4"
        sampleCount (int): 각 prompt마다 생성할 이미지 개수. Default: 10
        imageSize (tuple): 생성할 이미지 크기. Default: (512, 512)
        stepCount (int): sampling 과정의 inference step 수. Default: 100
        guidanceScale (float): classifier-free guidance 강도. Default: 7.5

    Returns:
        None
    """

    if modelVersion == "1.4": modelVersion = "CompVis/stable-diffusion-v1-4"
    elif modelVersion == "2.1": modelVersion = "stabilityai/stable-diffusion-2-1-base"
    
    device = "cuda:0" if torch.cuda.is_available() else "cpu"

    # modelVersion에 따라 모델 선택.
    model = StableDiffusionPipeline.from_pretrained(modelVersion).to(device)
    vae, tokenizer, textEncoder, unet = model.vae, model.tokenizer, model.text_encoder, model.unet

    scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

    promptDf = pd.read_csv(promptPath, index_col=0)
    # promptDf의 각 행 불러오기.
    for _, row in promptDf.iterrows():

        # B는 생성할 이미지 개수.
        B = sampleCount
        H, W = imageSize

        # classifier-free guidance를 위해 conditional과 uncoditional prompt가 필요함.
        prompts = [row.prompt] * B + [""] * B
        inputs = tokenizer(prompts, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids.to(device)
        # embds는 (2*B, 768, 77) 형태를 가지는 텍스트 임베딩.
        embds = textEncoder(inputs)[0]

        seed = row.evaluation_seed
        generator = torch.Generator()
        generator.manual_seed(seed)
        # 주어진 시드를 가지는 B개의 latent 생성.
        latents = torch.randn((B, unet.in_channels, H//8, W//8), generator=generator).to(device)
        latents *= scheduler.init_noise_sigma
        
        # inference step 수를 설정.
        scheduler.set_timesteps(stepCount)
        for t in tqdm.tqdm(scheduler.timesteps):

            # classifier-free guidance를 위해 uncoditional latent를 추가함.
            latentInputs = torch.cat([latents]*2)
            # latentInputs를 현재 timestep에 맞게 조정.
            latentInputs = scheduler.scale_model_input(latentInputs, timestep=t)

            # latent와 텍스트 임베딩을 사용하여 노이즈 생성.
            noises = unet(latentInputs, t, encoder_hidden_states=embds).sample
            # 앞쪽 B개의 노이즈는 conditional과, 뒤쪽 B개의 노이즈는 uncoditional.
            condNoises, uncondNoises = noises.chunk(2)
            # classifier-free guidance 적용.
            noises = uncondNoises + guidanceScale * (condNoises - uncondNoises)

            # 다음 latent 생성.
            latents = scheduler.step(noises, t, latents).prev_sample

        latents /= 0.18215
        # 최종 latent를 decoding하여 이미지 생성.
        images = vae.decode(latents).sample

        # images의 값을 [0,1]로 제한, 형태를 (B, C, H, W)에서 (B, H, W, C)로 변환.
        images = ((images + 1) / 2).clamp(0, 1).permute(0, 2, 3, 1)
        # numpy array 사용을 위해 cpu로 이동.
        images = images.detach().cpu().numpy()
        # [0,1] 범위의 float를 [0,255] 범위의 uint8로 변환.
        images = (images * 255).round().astype("uint8")
        # numpy array를 PIL Image로 변환.
        images = [Image.fromarray(image) for image in images]
        # images에 저장된 B개의 image 저장
        for idx, image in enumerate(images):
            image.save(f"image/{row.case_number}_{idx}.png")

if __name__ == "__main__":
    promptPath = "data/test_prompts.csv"
    generateImage(promptPath, sampleCount=2)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 100/100 [01:04<00:00,  1.55it/s]


In [ ]:
@torch.no_grad()
def generateImages(model, prompts, 
                   stepCount=100, guidanceScale=7.5,
                   path="image/erase"):
    
    """
    주어진 prompts에 해당하는 이미지 반환.

    Parameters:
        model (StableDiffusionPipeline): 이미지 생성시 사용할 모델
        prompts (List[str]): 이미지 생성시 사용할 프롬프트 string 리스트.
        sampleCount (int): prompt마다 생성할 이미지 개수. Default: 10
        imageSize (tuple): 생성할 이미지 크기. Default: (512, 512)
        stepCount (int): sampling 과정의 inference step 수. Default: 100
        guidanceScale (float): classifier-free guidance 강도. Default: 7.5

    Returns:
        
    """

    device = model.device
    vae, tokenizer, textEncoder, unet = model.vae, model.tokenizer, model.text_encoder, model.unet
    scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

    seeds = [random.randint(0, 5000) for idx in range(len(prompts))]
    paths = [f"{path}/{idx}.png" for idx in range(len(prompts))]

    for idx, prompt in enumerate(prompts):

        B = 1
        H, W = (512, 512)

        prompt = [prompt] * B + [""] * B
        input = tokenizer(prompt, padding="max_length", max_length=77, truncation=True, return_tensors="pt").input_ids.to(device)
        embd = textEncoder(input)[0]

        seed = seeds[idx]
        generator = torch.Generator()
        generator.manual_seed(seed)
        latent = torch.randn((B, unet.in_channels, H//8, W//8), generator=generator).to(device)
        latent *= scheduler.init_noise_sigma
        
        scheduler.set_timesteps(stepCount)
        for t in tqdm.tqdm(scheduler.timesteps):
            latentInput = torch.cat([latent] * 2)
            latentInput = scheduler.scale_model_input(latentInput, timestep=t)

            noise = unet(latentInput, t, encoder_hidden_states=embd).sample
            condNoise, uncondNoise = noise.chunk(2)
            noise = uncondNoise + guidanceScale * (condNoise - uncondNoise)

            latent = scheduler.step(noise, t, latent).prev_sample

        latent /= 0.18215
        image = vae.decode(latent).sample
        image = ((image + 1) / 2).clamp(0, 1).permute(0, 2, 3, 1)
        image = image.detach().cpu().numpy()
        image = (image * 255).round().astype("uint8")
        image = Image.fromarray(image[0])
        image.save(paths[idx])

    df = pd.DataFrame({"prompt": prompts, "seed": seeds, })